In [2]:
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import numpy as np

In [29]:
from szp_funcs import print_df, load_month, custom_create_res, months_year, get_otv, load_month_all, path_to_data, to_double

In [16]:
otv = get_otv()

In [18]:
otv[['inn', 'sch_name']]

,inn,sch_name
0,7743085543,ГБОУ Спецшкола № 1
1,7713229928,ГКОУ КШИ № 1
2,7704271031,ГБПОУ ПТ № 2
3,7714093099,ГКОУ СКОШИ № 2
4,7709438340,ГБПОУ КСУ № 3
...,...,...
935,7728113806,ИКИ РАН
936,7728013512,ИПУ РАН
937,7717067481,ГБОУ ШОР № 1
938,7729133548,СУНЦ МГУ


In [4]:
months = months_year[:5]
job = ['Педагог-психолог ', 'Старший педагог-психолог']

In [27]:
full = pd.read_excel(path_to_data + 'may.xlsx')

In [30]:
full['stv'] = full.apply(lambda row: to_double(row, 'stv'), axis=1)

In [33]:
print_df(pd.merge(full[full.job.isin(job)][['inn', 'snils', 'job', 'type', 'stv', 'status']], otv[['inn', 'sch_name']], how='left'), job[0] + ' за ' + months[-1])

In [5]:
res = custom_create_res(months, load_month)

In [6]:
def re_szp(row, months):
    active_months = 0
    fot = 0
    for month in months:
        if row['day_' + month] == 1 and row['stv_' + month] >= 1 and row['status_' + month] == row['status_pref_' + month] and row['status_' + month] == 'Работа':
            fot += row['sum_' + month]
            active_months += 1
    if active_months == 0:
        return np.nan
    return fot / active_months

In [7]:
def re_szp_job(row, months, jobs):
    active_months = 0
    fot = 0
    for month in months:
        if row['job_' + month] in jobs and row['day_' + month] == 1 and row['stv_' + month] >= 1 and row['status_' + month] == row['status_pref_' + month] and row['status_' + month] == 'Работа':
            fot += row['sum_' + month]
            active_months += 1
    if active_months == 0:
        return np.nan
    return fot / active_months

In [8]:
def re_fot(row, months):
    fot = 0
    for month in months:
        fot += row['sum_' + month]
    if fot == 0:
        return np.nan
    return fot

In [9]:
def re_fot_job(row, months, jobs):
    fot = 0
    for month in months:
        if row['job_' + month] in jobs:
            fot += row['sum_' + month]
    if fot == 0:
        return np.nan
    return fot

In [10]:
res['fot_job'] = res.apply(lambda row: re_fot_job(row, months, job), axis=1)
res['szp_job'] = res.apply(lambda row: re_szp_job(row, months, job), axis=1)

In [11]:
res

,inn,snils,sum_jan,job_jan,day_jan,type_jan,stv_jan,status_jan,status_pref_jan,sum_feb,...,job_may,day_may,type_may,stv_may,status_may,status_pref_may,wday_may,nday_may,fot_job,szp_job
0,5003021368,001-171-362 75,51468.76,Помощник воспитателя,0.0,Основное место работы,1.0,Работа,Работа,41289.47,...,Помощник воспитателя,1.0,Основное место работы,1.0,Работа,Работа,18.0,18.0,NaN,NaN
1,5003021368,001-191-281 85,106626.88,Учитель,1.0,Основное место работы,1.0,Работа,Работа,139753.92,...,Учитель,1.0,Основное место работы,1.0,Работа,Работа,18.0,18.0,NaN,NaN
2,5003021368,001-244-107 65,109394.87,Учитель,1.0,Основное место работы,1.0,Работа,Работа,109541.13,...,Учитель,0.0,Основное место работы,1.0,Работа,Работа,15.0,18.0,NaN,NaN
3,5003021368,001-263-234 77,74520.00,Воспитатель,1.0,Основное место работы,1.0,Работа,Работа,81178.00,...,Воспитатель,0.0,Основное место работы,1.0,Отпуск основной,Работа,16.0,18.0,NaN,NaN
4,5003021368,001-322-594 80,83013.52,Специалист,1.0,Основное место работы,1.0,Работа,Работа,75815.00,...,Специалист,0.0,Основное место работы,1.0,Работа,Работа,9.0,18.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176511,9718071371,152-940-312 49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Бухгалтер,1.0,Основное место работы,1.0,Работа,NaN,5.0,5.0,NaN,NaN
176512,9718071371,162-043-658 38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Учитель,0.0,Основное место работы,1.0,Работа,Работа,NaN,NaN,NaN,NaN
176513,9718071371,163-788-113 99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Бухгалтер,1.0,Основное место работы,1.0,Работа,NaN,11.0,11.0,NaN,NaN
176514,9718071371,185-458-671 20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Бухгалтер,1.0,Основное место работы,1.0,Работа,NaN,15.0,15.0,NaN,NaN


In [21]:
ans = res[['inn', 'snils', 'job_' + months[-1], 'fot_job', 'szp_job', 'stv_' + months[-1], 'status_' + months[-1], 'type_' + months[-1]]][res['job_' + months[-1]].isin(job)]#.groupby('inn').agg({
#     'fot_job': 'sum',
#     'szp_job': 'mean',
#     'stv_' + months[-1]: 'sum',
#     'snils': 'nunique'
# })

In [22]:
ans

,inn,snils,job_may,fot_job,szp_job,stv_may,status_may,type_may
73,5003021368,019-877-964 23,Педагог-психолог,400161.00,80032.20,1.0,Работа,Основное место работы
182,5003021368,048-922-073 76,Педагог-психолог,525544.61,109234.00,1.0,Работа,Основное место работы
264,5003021368,069-100-734 47,Педагог-психолог,550235.43,120369.25,1.0,Работа,Основное место работы
275,5003021368,070-945-203 58,Педагог-психолог,250297.11,NaN,1.0,Работа в отпуске по уходу за ребенком,Основное место работы
397,5003021368,112-438-858 43,Педагог-психолог,448394.96,93947.25,1.0,Работа,Основное место работы
...,...,...,...,...,...,...,...,...
175763,7733857776,174-760-708 93,Педагог-психолог,111313.05,57014.00,1.0,Работа,Основное место работы
175771,7734052005,197-575-235 30,Педагог-психолог,128417.62,NaN,1.0,NaN,Основное место работы
175782,7734233259,062-588-646 95,Педагог-психолог,126575.42,NaN,1.0,Работа,Основное место работы
176206,7726317748,038-722-370 62,Педагог-психолог,21739.13,NaN,1.0,Работа,Основное место работы


In [19]:
pd.merge(ans, otv[['inn', 'sch_name']], how='left')

,inn,snils,fot_job,szp_job,stv_may,status_may,type_may,sch_name
0,5003021368,019-877-964 23,400161.00,80032.20,1.0,Работа,Основное место работы,ГБОУ Школа № 1788
1,5003021368,048-922-073 76,525544.61,109234.00,1.0,Работа,Основное место работы,ГБОУ Школа № 1788
2,5003021368,069-100-734 47,550235.43,120369.25,1.0,Работа,Основное место работы,ГБОУ Школа № 1788
3,5003021368,070-945-203 58,250297.11,NaN,1.0,Работа в отпуске по уходу за ребенком,Основное место работы,ГБОУ Школа № 1788
4,5003021368,112-438-858 43,448394.96,93947.25,1.0,Работа,Основное место работы,ГБОУ Школа № 1788
...,...,...,...,...,...,...,...,...
3082,7733857776,174-760-708 93,111313.05,57014.00,1.0,Работа,Основное место работы,ГБОУ Школа № 2097
3083,7734052005,197-575-235 30,128417.62,NaN,1.0,NaN,Основное место работы,"ГБОУ ""Школа № 1212 Щукино"""
3084,7734233259,062-588-646 95,126575.42,NaN,1.0,Работа,Основное место работы,ГБОУ Школа № 1619
3085,7726317748,038-722-370 62,21739.13,NaN,1.0,Работа,Основное место работы,ГБУ ГППЦ ДОНМ


In [24]:
print_df(pd.merge(ans, otv[['inn', 'sch_name']], how='left'), job[0] + ' за ' + months[-1])

In [28]:
res[(res['job_' + months[-1]].isin(job)) & (res.inn == 7737078033)]#[['job_' + month for month in months]]

,inn,snils,sum_jan,job_jan,day_jan,type_jan,stv_jan,status_jan,status_pref_jan,sum_feb,...,job_apr,day_apr,type_apr,stv_apr,status_apr,status_pref_apr,wday_apr,nday_apr,fot_job,szp_job
172134,7737078033,153-358-303 56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29763.9,...,Юрисконсульт,1.0,Основное место работы,1.0,Работа,Работа,21.0,21.0,139144.62,54690.36


In [17]:
print_df(ans, job[0] + ' за ' + months[-1])